In [ ]:
import sys
# sys.path is a list of absolute path strings
sys.path.append('/mnt/sda/boris_stanoev/PropStar/')

In [ ]:
from normalization.normalize import Normalize
from sklearn.datasets import fetch_openml

In [ ]:
data = fetch_openml(data_id=1558)
denormalized_table = data.data.join(data.target)

In [ ]:
denormalized_table.columns.values

In [ ]:
denormalized_table.rename(columns={"V1": "age",
                                   "V2": "job",
                                   "V3": "marital",
                                   "V4": "education",
                                   "V5": "default",
                                   "V6": "balance",
                                   "V7": "housing",
                                   "V8": "loan",
                                   "V9": "contact",
                                   "V10": "day",
                                   "V11": "month",
                                   "V12": "duration",
                                   "V13": "campaign",
                                   "V14": "pdays",
                                   "V15": "previous",
                                   "V16": "poutcome",
                                  }, inplace=True)

In [ ]:
"""
bank client data:
    1 - age (numeric)
    
    2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur", "student","blue-collar","self-employed","retired","technician","services")
    
    3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)
    
    4 - education (categorical: "unknown","secondary","primary","tertiary")
    
    5 - default: has credit in default? (binary: "yes","no")
    
    6 - balance: average yearly balance, in euros (numeric)
    
    7 - housing: has housing loan? (binary: "yes","no")
    
    8 - loan: has personal loan? (binary: "yes","no")

related with the last contact of the current campaign:
    9 - contact: contact communication type (categorical: "unknown","telephone","cellular")
    
    10 - day: last contact day of the month (numeric)
    
    11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
    
    12 - duration: last contact duration, in seconds (numeric)

other attributes:
    13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
    
    14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
    
    15 - previous: number of contacts performed before this campaign and for this client (numeric)
    
    16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

output variable (desired target):
17 - y - has the client subscribed a term deposit? (binary: "yes","no")

"""

In [ ]:
denormalized_table

In [ ]:
norm = Normalize(denormalized_table=denormalized_table, denormalized_table_name="target")

In [ ]:
client_cols = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan']
norm.create_entity(source_entity_name="target", target_entity_name="client", columns=client_cols,
                   distinct_values=True)


norm.create_entity(source_entity_name="client", target_entity_name="default_type", columns=['default'],
                   distinct_values=True)
norm.create_entity(source_entity_name="client", target_entity_name="housing_type", columns=['housing'],
                   distinct_values=True)
norm.create_entity(source_entity_name="client", target_entity_name="loan_type", columns=['loan'],
                   distinct_values=True)

In [ ]:
contact_cols = ['contact', 'day', 'month', 'duration']
norm.create_entity(source_entity_name="target", target_entity_name="contact", columns=contact_cols,
                   distinct_values=True)
norm.create_entity(source_entity_name="contact", target_entity_name="contact_type", columns=['contact'],
                   distinct_values=True)

In [ ]:
attribute_cols = ['campaign', 'pdays', 'previous', 'poutcome']
norm.create_entity(source_entity_name="target", target_entity_name="attribute", columns=attribute_cols,
                   distinct_values=True)

norm.create_entity(source_entity_name="attribute", target_entity_name="previous", columns=['previous'],
                   distinct_values=True)

In [ ]:
norm.primary_keys
norm.relations
norm.entity_set.keys()


In [ ]:
for entity in norm.entity_set.keys():
    print(
        f"ALTER TABLE {entity} ADD CONSTRAINT PK_{entity} PRIMARY KEY ({norm.primary_keys[entity]});")

for parent_table in norm.relations:
    for child_table in norm.relations[parent_table]:
        print(f"ALTER TABLE {child_table} ADD CONSTRAINT FK_{child_table}_{parent_table} "
              f"FOREIGN KEY ({norm.relations[parent_table][child_table][1]}) "
              f"REFERENCES {parent_table}({norm.relations[parent_table][child_table][0]}));")


In [ ]:
norm.persist_entity_set(database_name="bank_marketing_exp2")